<a href="https://colab.research.google.com/github/cloud-vinny/Computing_Tech_Proj_Assignment/blob/main/SPAM_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Linear Logistic Regression**

**Indenpendencies**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, f1_score, recall_score, roc_auc_score, classification_report, adjusted_rand_score, normalized_mutual_info_score, silhouette_score
from scipy.stats import mode
from imblearn.over_sampling import RandomOverSampler

import string
import nltk
from nltk.stem import PorterStemmer
nltk.download('punkt_tab')
nltk.download('stopwords')
ps = PorterStemmer()

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**Re-clean the Preprocessed Data**

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/dataset_spam/cleaned_dataset.csv")
df.dropna(subset=['preprocessed_text'], inplace=True)
df.duplicated().sum()
df.isnull().sum()

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/dataset_spam/cleaned_dataset.csv'

**Transformation**

In [ ]:
ros = RandomOverSampler(random_state=2)
tfidf = TfidfVectorizer(max_features=3000, ngram_range=(1,2))
X = tfidf.fit_transform(df['preprocessed_text']).toarray()
y = df['spam'].values

def text_transform(text):
  text = text.lower() #lowercase
  text = nltk.word_tokenize(text) #tokenize

  # remove special chars & stopwords & punctuation & stemming
  b = []
  for a in text:
    if a.isalnum() and a not in stopwords.words('english') and a not in string.punctuation:
      b.append(ps.stem(a))

  return " ".join(b)

**Split dataset**

In [ ]:
X_train, X_test, y_train , y_test = train_test_split(X,y,test_size=0.25, random_state=42)
X_train_bal, y_train_bal = ros.fit_resample(X_train, y_train)

**Model training**

In [ ]:
clf = LogisticRegression(max_iter=300, class_weight='balanced', n_jobs=None)
clf.fit(X_train, y_train)

**Evaluation**

In [ ]:
y_pred2 = clf.predict(X_test)
y_prob2 = clf.predict_proba(X_test)[:,1]

# Evaluate (same metrics)
print("Accuracy      :", accuracy_score(y_test, y_pred2))
print("Precision     :", precision_score(y_test, y_pred2))
print("Recall        :", recall_score(y_test, y_pred2))
print("F1-Score      :", f1_score(y_test, y_pred2))
print("ROC-AUC       :", roc_auc_score(y_test, y_prob2))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred2))

**User INPUT to Check Spam Detection**

Replace the text in **mess="*text*"** of your choice to check the spam and non spam

In [ ]:
#=================================
# Replace here

mess = "Hey, are we still meeting for lunch tomorrow at 1 PM? Let me know if that works for you."

#=================================
print(f'Message: \"{mess}\"')
mess_clean = text_transform(mess)
mess_vec = tfidf.transform([mess_clean])
prediction = clf.predict(mess_vec)[0]
proba = clf.predict_proba(mess_vec)[0]
print("=================================")
print(f"Probability of NON-SPAM: {proba[0]*100:.2f}%")
print(f"Probability of SPAM    : {proba[1]*100:.2f}%")
print("=================================")
if prediction == 1:
    print("Predicted label: SPAM")
else:
    print("Predicted label: NON-SPAM")